In [9]:
# ✅ STEP 1: Mount Google Drive and configure environment
from google.colab import drive
drive.mount('/content/drive')

# ✅ STEP 2: Install and import required libraries with error handling
try:
    !pip install -q pandas sqlalchemy openpyxl tqdm
    print("✓ All packages installed successfully")
except Exception as e:
    print(f"⚠️ Issue with package installation: {e}")

import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import os
from datetime import datetime
from tqdm.notebook import tqdm
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger('layoffs_cleaning')

# ✅ STEP 3: Define paths with improved error handling
try:
    input_path = '/content/drive/My Drive/Data_Analyst/Portfolio_Projects/Projects/SQL/Layoffs_Dataset_for_Data_Cleaning.csv'
    output_dir = '/content/drive/My Drive/Data_Analyst/Portfolio_Projects/Projects/SQL'

    # Create timestamped output filename for versioning
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_excel = os.path.join(output_dir, f'Layoffs_Dataset_Cleaned_{timestamp}.xlsx')
    output_csv = os.path.join(output_dir, f'Layoffs_Dataset_Cleaned_{timestamp}.csv')

    # Check if input file exists
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Input file not found: {input_path}")

    logger.info(f"Input file confirmed: {input_path}")
except Exception as e:
    logger.error(f"Path setup error: {e}")
    raise

# ✅ STEP 4: Load initial data to get column names and structure
try:
    logger.info("Loading raw data sample...")
    # Just load a small sample to get schema
    df_sample = pd.read_csv(input_path, nrows=5)
    logger.info(f"Raw data columns: {', '.join(df_sample.columns)}")
except Exception as e:
    logger.error(f"Data loading error: {e}")
    raise

# ✅ STEP 5: Create SQL engine and load data directly using SQL
try:
    logger.info("Creating SQL engine...")
    engine = create_engine('sqlite://', echo=False)

    # Create table structure first
    with engine.begin() as conn:
        conn.execute(text("""
            CREATE TABLE layoffs_raw (
                company TEXT,
                location TEXT,
                industry TEXT,
                total_laid_off TEXT,
                percentage_laid_off TEXT,
                date TEXT,
                stage TEXT,
                country TEXT,
                funds_raised TEXT
            )
        """))

    # Load data in chunks using SQL COPY or equivalent
    logger.info("Loading data into SQLite...")
    chunksize = 10000
    for chunk in tqdm(pd.read_csv(input_path, chunksize=chunksize)):
        chunk.to_sql('layoffs_raw', con=engine, if_exists='append', index=False)

    # Get raw counts for reporting
    with engine.connect() as conn:
        raw_count = conn.execute(text("SELECT COUNT(*) FROM layoffs_raw")).scalar()

    logger.info(f"Raw data loaded: {raw_count} rows")

    # Get missing values using SQL
    with engine.connect() as conn:
        missing_columns = conn.execute(text("""
            SELECT
                SUM(CASE WHEN company IS NULL OR company = '' THEN 1 ELSE 0 END) AS company_missing,
                SUM(CASE WHEN location IS NULL OR location = '' THEN 1 ELSE 0 END) AS location_missing,
                SUM(CASE WHEN industry IS NULL OR industry = '' THEN 1 ELSE 0 END) AS industry_missing,
                SUM(CASE WHEN total_laid_off IS NULL OR total_laid_off = '' THEN 1 ELSE 0 END) AS total_laid_off_missing,
                SUM(CASE WHEN percentage_laid_off IS NULL OR percentage_laid_off = '' THEN 1 ELSE 0 END) AS percentage_laid_off_missing,
                SUM(CASE WHEN date IS NULL OR date = '' THEN 1 ELSE 0 END) AS date_missing,
                SUM(CASE WHEN stage IS NULL OR stage = '' THEN 1 ELSE 0 END) AS stage_missing,
                SUM(CASE WHEN country IS NULL OR country = '' THEN 1 ELSE 0 END) AS country_missing,
                SUM(CASE WHEN funds_raised IS NULL OR funds_raised = '' THEN 1 ELSE 0 END) AS funds_raised_missing
            FROM layoffs_raw
        """)).fetchone()

    missing_stats = {col: val for col, val in zip(df_sample.columns, missing_columns)}
    for col, val in missing_stats.items():
        logger.info(f"Column '{col}': {val} missing values ({val/raw_count*100:.2f}%)")

    # Find duplicates using SQL - FIXED: SQLite doesn't support COUNT(DISTINCT multiple columns)
    with engine.connect() as conn:
        # Alternative approach: Count total rows vs unique combinations using GROUP BY
        total_rows = conn.execute(text("SELECT COUNT(*) FROM layoffs_raw")).scalar()

        # Count unique combinations
        unique_rows = conn.execute(text("""
            SELECT COUNT(*) FROM (
                SELECT company, location, industry, total_laid_off,
                       percentage_laid_off, date, stage, country, funds_raised
                FROM layoffs_raw
                GROUP BY company, location, industry, total_laid_off,
                         percentage_laid_off, date, stage, country, funds_raised
            )
        """)).scalar()

        dup_count = total_rows - unique_rows

    logger.info(f"Found {dup_count} potential duplicate rows ({dup_count/raw_count*100:.2f}%)")

except Exception as e:
    logger.error(f"SQL engine error: {e}")
    raise

# ✅ STEP 6: Comprehensive SQL data cleaning pipeline
try:
    logger.info("Starting SQL cleaning process...")
    with engine.begin() as conn:
        # Step 6.1: Create a table with basic cleaning and type conversions
        logger.info("Creating initial clean table...")
        conn.execute(text("""
            CREATE TABLE layoffs_base AS
            SELECT
                TRIM(company) AS company,
                TRIM(location) AS location,
                TRIM(LOWER(REPLACE(REPLACE(industry, ' ', '_'), '-', '_'))) AS industry,
                CASE
                    WHEN total_laid_off = '' THEN NULL
                    ELSE CAST(total_laid_off AS INTEGER)
                END AS total_laid_off,
                CASE
                    WHEN percentage_laid_off = '' THEN NULL
                    WHEN percentage_laid_off LIKE '%\%%' THEN
                        CAST(REPLACE(percentage_laid_off, '%', '') AS REAL)
                    WHEN CAST(percentage_laid_off AS REAL) > 1.0 AND CAST(percentage_laid_off AS REAL) <= 100 THEN
                        CAST(percentage_laid_off AS REAL)
                    WHEN CAST(percentage_laid_off AS REAL) > 0 AND CAST(percentage_laid_off AS REAL) < 1.0 THEN
                        CAST(percentage_laid_off AS REAL) * 100
                    ELSE CAST(percentage_laid_off AS REAL)
                END AS percentage_laid_off,
                date,
                TRIM(stage) AS stage,
                TRIM(country) AS country,
                CASE
                    WHEN funds_raised = '' THEN NULL
                    ELSE CAST(funds_raised AS REAL)
                END AS funds_raised
            FROM layoffs_raw
            WHERE NOT (
                (company IS NULL OR company = '') AND
                (location IS NULL OR location = '') AND
                (industry IS NULL OR industry = '')
            )
        """))

        # Step 6.2: Create a table with deduplicated records
        logger.info("Removing duplicates...")
        conn.execute(text("""
            CREATE TABLE layoffs_deduped AS
            WITH numbered_rows AS (
                SELECT *,
                    ROW_NUMBER() OVER (
                        PARTITION BY company, location, industry, total_laid_off,
                                    percentage_laid_off, date, stage, country, funds_raised
                        ORDER BY company
                    ) AS rn
                FROM layoffs_base
            )
            SELECT
                company, location, industry, total_laid_off, percentage_laid_off,
                date, stage, country, funds_raised
            FROM numbered_rows
            WHERE rn = 1
        """))

        # Step 6.3: Create a table with standardized values
        logger.info("Standardizing values...")
        conn.execute(text("""
            CREATE TABLE layoffs_normalized AS
            SELECT
                CASE
                    WHEN company = 'Intel' THEN 'Intel Corporation'
                    WHEN company = 'Amazon' THEN 'Amazon.com Inc.'
                    WHEN company = 'Meta' THEN 'Meta Platforms Inc.'
                    WHEN company = 'Google' THEN 'Google LLC'
                    ELSE company
                END AS company,

                location,

                CASE
                    WHEN industry IN ('cryptocurrency', 'crypto_currency', 'crypto') THEN 'crypto'
                    WHEN industry IN ('consumer', 'consumer_retail', 'consumer_services') THEN 'consumer_goods'
                    WHEN industry IN ('healthcare', 'health', 'medical') THEN 'health_care'
                    WHEN industry IN ('finance', 'financial') THEN 'financial_services'
                    WHEN industry IN ('transport', 'transportation') THEN 'transportation'
                    WHEN industry IN ('realestate', 'real_estate', 'property') THEN 'real_estate'
                    WHEN industry IN ('ai', 'artificial_intelligence', 'machine_learning') THEN 'artificial_intelligence'
                    WHEN industry IN ('saas', 'software_as_a_service') THEN 'saas'
                    ELSE industry
                END AS industry,

                total_laid_off,
                percentage_laid_off,

                -- Validate date format
                CASE
                    WHEN date IS NULL OR date = '' THEN NULL
                    WHEN date LIKE '____-__-__' THEN date  -- Already YYYY-MM-DD
                    -- Add more date format conversions as needed
                    ELSE date
                END AS date,

                CASE
                    WHEN stage IS NULL OR stage = '' THEN 'unknown'
                    WHEN stage IN ('series c-d+', 'series c-d', 'series c-d plus', 'c-d') THEN 'series c-d'
                    WHEN stage IN ('private', 'privately held') THEN 'private equity'
                    WHEN stage IN ('series a-b', 'series a-b+', 'a-b') THEN 'series a-b'
                    WHEN stage = 'ipo' THEN 'public'
                    ELSE stage
                END AS stage,

                CASE
                    WHEN country IS NULL OR country = '' THEN 'unknown'
                    WHEN country IN ('united states', 'usa', 'us', 'u.s.', 'u.s.a.') THEN 'USA'
                    WHEN country IN ('united kingdom', 'uk', 'u.k.', 'britain', 'great britain') THEN 'UK'
                    WHEN country IN ('uae', 'united arab emirates') THEN 'UAE'
                    -- SQLite doesn't have INITCAP, so handle common countries
                    WHEN LOWER(country) = 'canada' THEN 'Canada'
                    WHEN LOWER(country) = 'germany' THEN 'Germany'
                    WHEN LOWER(country) = 'france' THEN 'France'
                    WHEN LOWER(country) = 'india' THEN 'India'
                    WHEN LOWER(country) = 'china' THEN 'China'
                    WHEN LOWER(country) = 'australia' THEN 'Australia'
                    WHEN LOWER(country) = 'japan' THEN 'Japan'
                    ELSE country
                END AS country,

                funds_raised
            FROM layoffs_deduped
        """))

        # Step 6.4: Create final table with additional filters and validations
        logger.info("Creating final cleaned table...")
        conn.execute(text("""
            CREATE TABLE layoffs_cleaned AS
            SELECT *
            FROM layoffs_normalized
            WHERE (company IS NOT NULL AND company != '')
              AND (date IS NOT NULL AND date != '')
              AND NOT (total_laid_off IS NULL AND percentage_laid_off IS NULL)
        """))

        # Step 6.5: Add derived columns using SQL
        logger.info("Adding derived columns...")
        conn.execute(text("""
            CREATE TABLE layoffs_final AS
            SELECT
                company,
                location,
                industry,
                total_laid_off,
                percentage_laid_off,
                date,
                -- Extract year, month, quarter using SQLite functions
                CAST(SUBSTR(date, 1, 4) AS INTEGER) AS year,
                CAST(SUBSTR(date, 6, 2) AS INTEGER) AS month,
                CASE
                    WHEN CAST(SUBSTR(date, 6, 2) AS INTEGER) BETWEEN 1 AND 3 THEN 1
                    WHEN CAST(SUBSTR(date, 6, 2) AS INTEGER) BETWEEN 4 AND 6 THEN 2
                    WHEN CAST(SUBSTR(date, 6, 2) AS INTEGER) BETWEEN 7 AND 9 THEN 3
                    WHEN CAST(SUBSTR(date, 6, 2) AS INTEGER) BETWEEN 10 AND 12 THEN 4
                    ELSE NULL
                END AS quarter,
                stage,
                country,
                funds_raised,
                -- Add layoff severity category
                CASE
                    WHEN percentage_laid_off <= 5 THEN 'Very Low'
                    WHEN percentage_laid_off <= 10 THEN 'Low'
                    WHEN percentage_laid_off <= 25 THEN 'Medium'
                    WHEN percentage_laid_off <= 50 THEN 'High'
                    ELSE 'Very High'
                END AS layoff_severity
            FROM layoffs_cleaned
            ORDER BY date
        """))

        # Step 6.6: Create summary statistics table for reporting
        logger.info("Creating summary statistics table...")
        conn.execute(text("""
            CREATE TABLE layoffs_summary AS
            SELECT
                COUNT(DISTINCT company) AS total_companies,
                COUNT(DISTINCT industry) AS total_industries,
                COUNT(DISTINCT country) AS total_countries,
                MIN(date) AS earliest_date,
                MAX(date) AS latest_date,
                SUM(total_laid_off) AS total_employees_laid_off,
                AVG(percentage_laid_off) AS avg_percentage_laid_off,
                MAX(percentage_laid_off) AS max_percentage_laid_off,
                SUM(funds_raised) AS total_funds_raised,
                COUNT(*) AS total_layoff_events,
                (SELECT COUNT(*) FROM layoffs_raw) AS original_row_count
            FROM layoffs_final
        """))

        # Step 6.7: Create industry summary table
        logger.info("Creating industry summary table...")
        conn.execute(text("""
            CREATE TABLE industry_summary AS
            SELECT
                industry,
                COUNT(*) AS layoff_events,
                SUM(total_laid_off) AS total_laid_off,
                AVG(percentage_laid_off) AS avg_percentage_laid_off,
                COUNT(DISTINCT company) AS companies_affected
            FROM layoffs_final
            WHERE industry IS NOT NULL AND industry != ''
            GROUP BY industry
            ORDER BY total_laid_off DESC
        """))

        # Step 6.8: Create country summary table
        logger.info("Creating country summary table...")
        conn.execute(text("""
            CREATE TABLE country_summary AS
            SELECT
                country,
                COUNT(*) AS layoff_events,
                SUM(total_laid_off) AS total_laid_off,
                AVG(percentage_laid_off) AS avg_percentage_laid_off,
                COUNT(DISTINCT company) AS companies_affected
            FROM layoffs_final
            WHERE country IS NOT NULL AND country != ''
            GROUP BY country
            ORDER BY total_laid_off DESC
        """))

        # Step 6.9: Create time trend summary
        logger.info("Creating time trend summary...")
        conn.execute(text("""
            CREATE TABLE time_summary AS
            SELECT
                year,
                quarter,
                COUNT(*) AS layoff_events,
                SUM(total_laid_off) AS total_laid_off,
                AVG(percentage_laid_off) AS avg_percentage_laid_off,
                COUNT(DISTINCT company) AS companies_affected
            FROM layoffs_final
            GROUP BY year, quarter
            ORDER BY year, quarter
        """))

    # Get counts after each step for reporting
    with engine.connect() as conn:
        base_count = conn.execute(text("SELECT COUNT(*) FROM layoffs_base")).scalar()
        deduped_count = conn.execute(text("SELECT COUNT(*) FROM layoffs_deduped")).scalar()
        final_count = conn.execute(text("SELECT COUNT(*) FROM layoffs_final")).scalar()

    logger.info(f"Initial cleaning: {raw_count} -> {base_count} rows")
    logger.info(f"After deduplication: {base_count} -> {deduped_count} rows")
    logger.info(f"Final dataset: {deduped_count} -> {final_count} rows")

except Exception as e:
    logger.error(f"SQL transformation error: {e}")
    raise

# ✅ STEP 7: Export results using SQL to extract data
try:
    logger.info("Exporting final cleaned dataset...")

    # Get the main cleaned dataset
    df_final = pd.read_sql_query("SELECT * FROM layoffs_final", engine)

    # Convert date to proper datetime AND ensure it's timezone-unaware
    df_final['date'] = pd.to_datetime(df_final['date'], errors='coerce').dt.tz_localize(None)

    # Get summary tables
    df_summary = pd.read_sql_query("SELECT * FROM layoffs_summary", engine)
    df_industry = pd.read_sql_query("SELECT * FROM industry_summary", engine)
    df_country = pd.read_sql_query("SELECT * FROM country_summary", engine)
    df_time = pd.read_sql_query("SELECT * FROM time_summary", engine)

    # Make sure any date columns in summary tables are also timezone-unaware
    if 'earliest_date' in df_summary.columns:
        df_summary['earliest_date'] = pd.to_datetime(df_summary['earliest_date'], errors='coerce').dt.tz_localize(None)
    if 'latest_date' in df_summary.columns:
        df_summary['latest_date'] = pd.to_datetime(df_summary['latest_date'], errors='coerce').dt.tz_localize(None)

    # Format summary table for display
    summary_stats = pd.DataFrame({
        'Metric': [
            'Total Companies',
            'Total Industries',
            'Total Countries',
            'Earliest Date',
            'Latest Date',
            'Total Employees Laid Off',
            'Average % Laid Off',
            'Max % Laid Off',
            'Total Funds Raised (Millions)',
            'Total Layoff Events',
            'Original Row Count',
            'Data Reduction %'
        ],
        'Value': [
            df_summary['total_companies'].iloc[0],
            df_summary['total_industries'].iloc[0],
            df_summary['total_countries'].iloc[0],
            df_summary['earliest_date'].iloc[0],
            df_summary['latest_date'].iloc[0],
            f"{df_summary['total_employees_laid_off'].iloc[0]:,.0f}",
            f"{df_summary['avg_percentage_laid_off'].iloc[0]:.2f}%",
            f"{df_summary['max_percentage_laid_off'].iloc[0]:.2f}%",
            f"${df_summary['total_funds_raised'].iloc[0]:,.2f}",
            df_summary['total_layoff_events'].iloc[0],
            df_summary['original_row_count'].iloc[0],
            f"{(1 - df_summary['total_layoff_events'].iloc[0]/df_summary['original_row_count'].iloc[0])*100:.2f}%"
        ]
    })

    # Save to Excel with multiple sheets
    logger.info(f"Saving to Excel: {output_excel}")
    with pd.ExcelWriter(output_excel, engine='openpyxl') as writer:
        df_final.to_excel(writer, sheet_name='Cleaned_Data', index=False)
        summary_stats.to_excel(writer, sheet_name='Summary', index=False)
        df_industry.to_excel(writer, sheet_name='Industry_Analysis', index=False)
        df_country.to_excel(writer, sheet_name='Country_Analysis', index=False)
        df_time.to_excel(writer, sheet_name='Time_Analysis', index=False)

    # Save to CSV for easier machine processing
    logger.info(f"Saving to CSV: {output_csv}")
    df_final.to_csv(output_csv, index=False)

    logger.info("✅ Files saved successfully")

except Exception as e:
    logger.error(f"Export error: {e}")
    raise

# ✅ STEP 8: Create additional views for analysis (pure SQL approach)
try:
    logger.info("Creating analysis views...")
    with engine.begin() as conn:
        # Top companies by layoff count
        conn.execute(text("""
            CREATE VIEW top_layoff_companies AS
            SELECT
                company,
                SUM(total_laid_off) AS total_employees_laid_off,
                COUNT(*) AS layoff_events,
                AVG(percentage_laid_off) AS avg_percentage_laid_off
            FROM layoffs_final
            GROUP BY company
            ORDER BY total_employees_laid_off DESC
            LIMIT 10
        """))

        # Layoff trends by quarter
        conn.execute(text("""
            CREATE VIEW quarterly_layoff_trends AS
            SELECT
                year,
                quarter,
                SUM(total_laid_off) AS total_laid_off,
                COUNT(*) AS layoff_events,
                COUNT(DISTINCT company) AS companies_affected
            FROM layoffs_final
            GROUP BY year, quarter
            ORDER BY year, quarter
        """))

        # Industry layoff patterns
        conn.execute(text("""
            CREATE VIEW industry_layoff_patterns AS
            SELECT
                industry,
                AVG(percentage_laid_off) AS avg_percentage_laid_off,
                SUM(total_laid_off) AS total_laid_off,
                COUNT(DISTINCT company) AS companies_affected,
                MIN(date) AS first_layoff,
                MAX(date) AS latest_layoff
            FROM layoffs_final
            GROUP BY industry
            ORDER BY total_laid_off DESC
        """))
except Exception as e:
    logger.error(f"Analysis views error: {e}")
    # Non-critical, continue execution

# ✅ STEP 9: Print summary statistics
print("\n" + "="*50)
print(f"✅ DATA CLEANING COMPLETE")
print("="*50)
print(f"🧼 Final dataset shape: {df_final.shape[0]} rows × {df_final.shape[1]} columns")
print(f"📊 Date range: {pd.to_datetime(df_summary['earliest_date'].iloc[0]).strftime('%Y-%m-%d')} to {pd.to_datetime(df_summary['latest_date'].iloc[0]).strftime('%Y-%m-%d')}")
print(f"🏢 Companies analyzed: {df_summary['total_companies'].iloc[0]}")
print(f"🌐 Countries represented: {df_summary['total_countries'].iloc[0]}")
print(f"🔢 Total employees laid off: {df_summary['total_employees_laid_off'].iloc[0]:,.0f}")
print(f"📄 Files saved:")
print(f"   - Excel: {os.path.basename(output_excel)}")
print(f"   - CSV: {os.path.basename(output_csv)}")
print("="*50)

# Return most interesting findings
top_industries = pd.read_sql_query("SELECT industry, total_laid_off FROM industry_summary ORDER BY total_laid_off DESC LIMIT 5", engine)
print("\n🔍 TOP 5 INDUSTRIES BY LAYOFFS:")
for i, row in top_industries.iterrows():
    print(f"   {i+1}. {row['industry']}: {row['total_laid_off']:,.0f} employees")

# Return the cleaned dataframe sample for inspection
df_final.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ All packages installed successfully


0it [00:00, ?it/s]


✅ DATA CLEANING COMPLETE
🧼 Final dataset shape: 1739 rows × 13 columns
📊 Date range: 2023-01-01 to 2025-03-19
🏢 Companies analyzed: 1383
🌐 Countries represented: 47
🔢 Total employees laid off: 440,074
📄 Files saved:
   - Excel: Layoffs_Dataset_Cleaned_20250406_134946.xlsx
   - CSV: Layoffs_Dataset_Cleaned_20250406_134946.csv

🔍 TOP 5 INDUSTRIES BY LAYOFFS:
   1. other: 56,815 employees
   2. hardware: 55,823 employees
   3. consumer_goods: 45,777 employees
   4. retail: 43,067 employees
   5. transportation: 32,395 employees


,company,location,industry,total_laid_off,percentage_laid_off,date,year,month,quarter,stage,country,funds_raised,layoff_severity
0,Micron,['Boise'],hardware,4800.0,10.0,2023-01-01,2023,1,1,Post-IPO,United States,50.0,Low
1,Amdocs,['St. Louis'],support,700.0,3.0,2023-01-02,2023,1,1,Post-IPO,United States,NaN,Very Low
2,Bytedance,"['Shanghai', 'Non-U.S.']",consumer_goods,NaN,10.0,2023-01-03,2023,1,1,Unknown,China,9400.0,Low
3,Harappa,"['New Delhi', 'Non-U.S.']",education,60.0,30.0,2023-01-03,2023,1,1,Acquired,India,NaN,High
4,Pegasystems,['Boston'],hr,245.0,4.0,2023-01-03,2023,1,1,Post-IPO,United States,NaN,Very Low
